In [1]:
!pip install -U accelerate bitsandbytes datasets peft transformers tokenizers >> /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 101.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully uninstalled transformers-4.37.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-

This section aims to guide you through converting these datasets into a uniform format compatible with training regimens. To prepare for training, one must select a format.

### Targeting ChatML Format as Below:

<|im_start|>system
You are an AI assistant. User will you give you a task. Your goal is to 
complete the task as faithfully as you can. While performing the task 
think step-by-step and justify your steps.<|im_end|>

<|im_start|>user
Premise: A man is inline skating in front of a wooden bench. Hypothesis: 
A man is having fun skating in front of a bench. .Choose the correct 
answer: Given the premise, can we conclude the hypothesis? 
Select from: a). yes b). it is not possible to tell c). no<|im_end|>

<|im_start|>assistant
b). it is not possible to tell Justification: Although the man is inline 
skating in front of the wooden bench, we cannot conclude whether he is 
having fun or not, as his emotions are not explicitly mentioned.<|im_end|>

In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# Working on Open-Assistant Dataset

from datasets import load_dataset

dataset = load_dataset("OpenAssistant/oasst_top1_2023-08-25")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
print(dataset["train"][0]["text"])

<|im_start|>user
Consigliami 5 nomi per il mio cucciolo di dobberman<|im_end|>
<|im_start|>assistant
Ecco 5 nomi per il tuo cucciolo di dobermann:

- Zeus
- Apollo
- Thor
- Athena
- Odin<|im_end|>



In [7]:
orcaset = load_dataset("Open-Orca/OpenOrca")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
orcaset

DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 4233923
    })
})

In [9]:
orcaset = orcaset['train'].train_test_split(test_size=0.1)
orcaset

DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 3810530
    })
    test: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 423393
    })
})

In [11]:
orcaset['train'][0]

{'id': 't0.839935',
 'system_prompt': 'You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.',
 'question': "Answer the following question: Fact 1: The color of leaves on deciduous trees usually changes in the autumn season.  Fact 2: Common deciduous trees are the oak, elm and beech.  Given the two facts above, which tree's leaves change color in autumn?\nAnswer:",
 'response': 'Based on the given facts, the leaves of deciduous trees change color in the autumn season. Common deciduous trees include the oak, elm, and beech. Therefore, the leaves of all three of these trees change color in autumn.'}

In [13]:
def format_conversation(row):
    template="<|im_start|>system\n{sys}<|im_end|>\n<|im_start|>user\n{q}<|im_end|>\n<|im_start|>assistant\n{a}<|im_end|>"
    # Above template contains the 3 variable that will be filled up in the format method
    conversation=template.format(
        sys=row["system_prompt"],
        q=row["question"],
        a=row["response"],
    )

    return {"text": conversation}

In [16]:
import os
orcaset = orcaset.map(
    format_conversation,
    remove_columns=orcaset['train'].column_names,
    num_proc=os.cpu_count()
)

Map (num_proc=4):   0%|          | 0/3810530 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/423393 [00:00<?, ? examples/s]

In [17]:
dataset = load_dataset("g-ronimo/lfpodcast")
dataset = dataset["train"].train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
import json
print(json.dumps(dataset["train"][0],indent=2))

{
  "token_count": 18435,
  "char_count": 79317,
  "conversation": [
    {
      "from": "Guest",
      "text": "So I do, especially when there's turbulence, which is, you know, like on the way here, there was turbulence and the plane jiggled. Even the flight attendant had to hold on to the side. And I was just thinking to myself, it's amazing that this happens all the time and the wings don't fall off. You know, given how many planes are flying. But then I often think about it and I'm like, you know, a long time ago, I think people didn't trust elevators in a 40-story building in New York City. And now we just take it completely for granted that you can step into this shaft, which is 40 floors up and down, and it will just not fail."
    },
    {
      "from": "Lex",
      "text": "Yeah, again, I'm the same way with elevators, but also buildings. When I'll stand on the 40th floor and wonder how the heck are we not falling right now? Like how amazing it is with the high winds, like str

In [20]:
def format_conversation(row):
    # Template for conversation turns in ChatML format
    template="<|im_start|>user\n{q}<|im_end|>\n<|im_start|>assistant\n{a}<|im_end|>"

    turns=row["conversation"]

    # If Lex is the first speaker, skip his turn to start with Guest's question
    if turns[0]["from"]=="Lex":  
        turns=turns[1:]

    conversation=[]
    for i in range(0, len(turns), 2):
        # Assuming the conversation always alternates between Guest and Lex
        question=turns[i]  # Guest
        answer=turns[i+1]  # Lex

        conversation.append(
            template.format(
                q=question["text"],
                a=answer["text"],
                ))
    return {"text": "\n".join(conversation)}

import os
dataset = dataset.map(
    format_conversation, 
    remove_columns=dataset["train"].column_names,
    num_proc=os.cpu_count()  
)

Map (num_proc=4):   0%|          | 0/327 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/37 [00:00<?, ? examples/s]

> mv anthony-bourdain-kitchen-confidential.pdf kc.pdf

> pdftotext  -nopgbrk kc.pdf 

#### fix line breaks within sentence

> sed -r ':a /[a-zA-Z,\ ]$/N;s/(.)\n/\1 /;ta' kc.txt > kc_reformat.txt

In [ ]:
# Gather paragraphs to target
with open("kc_reformat.txt") as f:
    file_content = f.read()

chapters=file_content.split("\n\n")

# Define minimum and maximum lengths to ensure a good interview flow
passage_minlen=300  # if paragraph <300 chars -> merge with next
passage_maxlen=2000  # if paragraph >2k chars -> split

# Process the chapters into suitable interview passages
passages=[]
for chap in chapters:
    passage=""
    for par in chap.split("\n"):
        if(len(passage)<passage_minlen) or not passage[-1]=="." and len(passage)<passage_maxlen:
            passage+="\n" + par
        else:
            passages.append(passage.strip().replace("\n", " "))
            passage=par
      
# Ask Open Hermes
prompt_template="""<|im_start|>system
You are an expert interviewer who interviews an autobiography of a famous chef. 
You formulate questions based on quotes from the autobiography. Below is one 
such quote. Formulate a question that the quote would be the perfect answer to. 
The question should be short and directed at the author of the autobiography 
like in an interview. The question is short. Remember, make the question as 
short as possible. Do not give away the answer in your question. 
Also: If possible, ask for motvations, feelings, and perceptions rather than 
events or facts.

Here is some context that might help you formulate the question regarding the quote:
{ctx}
<|im_end|>
<|im_start|>user
Quote:
{par}<|im_end|>
<|im_start|>assistant
Question:"""

prompts=[]
for i,p in enumerate(passages):
    prompt=prompt_template.format(par=passages[i], ctx=passages[i-1]) 
    prompts.append(prompt)

# Prompt smart LLM, parse results, store Q/A in .json  
...

In [33]:
del model

In [25]:
# load and prepare dataset

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

modelpath="mistralai/Mistral-7B-v0.1"

# Load 4-bit quantized model
model = AutoModelForCausalLM.from_pretrained(
    modelpath,    
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    ),
    torch_dtype=torch.bfloat16,
)

# Load (slow) Tokenizer, fast tokenizer sometimes ignores added tokens
tokenizer = AutoTokenizer.from_pretrained(modelpath, use_fast=False)   

# Add tokens <|im_start|> and <|im_end|>, latter is special eos token 
tokenizer.pad_token = "</s>"
tokenizer.add_tokens(["<|im_start|>"])
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))
model.resize_token_embeddings(len(tokenizer))
model.config.eos_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

we find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and that LoRA on all linear transformer block layers are required to match full finetuning performance

LoRA rank r: Determines the size of the low-rank matrices. The higher the rank the more parameters you train and the bigger your adapter files will be. Usually a number between 8 and 128. The maximum possible value, ie. training all parameters, would be 4096 for llama2-7b and Mistral (=hidden_size in config.json) and defeat the purpose of adding adapters. The QLoRA paper suggests 64 for Guanaco (Open Assistant dataset) which works well for me.

target_modules: Another suggestion/finding of the QLoRA authors in their paper:

modules_to_save: Specifies the modules apart from the LoRA layers to be set as trainable and saved in the final checkpoint. Since we added the ChatML tags as tokens to the vocabulary, we need to train and save the linear layer lm_head and the embedding matrix embed_tokens too. This will be relevant for merging the adapter back into the base model later.

In [26]:
# Add LoRA adapters to model
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=64, 
    lora_alpha=16, 
    target_modules = ['q_proj', 'k_proj', 'down_proj', 'v_proj', 'gate_proj', 'o_proj', 'up_proj'],
    lora_dropout=0.1, 
    bias="none", 
    modules_to_save = ["lm_head", "embed_tokens"],        # needed because we added new tokens to tokenizer/model
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)
model.config.use_cache = False

In [27]:
def tokenize(element):
    return tokenizer(
        element["text"],
        truncation=True,
        max_length=2048,
        add_special_tokens=False,
    )

dataset_tokenized = dataset.map(
    tokenize, 
    batched=True, 
    num_proc=os.cpu_count(),    # multithreaded
    remove_columns=["text"]     # don't need the strings anymore, we have tokens from here on
)

Map (num_proc=4):   0%|          | 0/327 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/37 [00:00<?, ? examples/s]

The Hugging Face trainer expects a collator function to transform a list of samples to a dictionary holding a batch of padded

input_ids (tokenized text)

labels (target text, same as input_ids)

and attention_masks (tensor of zeros and ones).

In [28]:
# collate function - to transform list of dictionaries [ {input_ids: [123, ..]}, {.. ] to single batch dictionary { input_ids: [..], labels: [..], attention_mask: [..] }
def collate(elements):
    tokenlist=[e["input_ids"] for e in elements]
    tokens_maxlen=max([len(t) for t in tokenlist])  # length of longest input

    input_ids,labels,attention_masks = [],[],[]
    for tokens in tokenlist:
        # how many pad tokens to add for this sample
        pad_len=tokens_maxlen-len(tokens)

        # pad input_ids with pad_token, labels with ignore_index (-100) and set attention_mask 1 where content, otherwise 0
        input_ids.append( tokens + [tokenizer.pad_token_id]*pad_len )   
        labels.append( tokens + [-100]*pad_len )    
        attention_masks.append( [1]*len(tokens) + [0]*pad_len ) 

    batch={
        "input_ids": torch.tensor(input_ids),
        "labels": torch.tensor(labels),
        "attention_mask": torch.tensor(attention_masks)
    }
    return batch

In [31]:
bs=4        # batch size
ga_steps=1  # gradient acc. steps
epochs=5
steps_per_epoch=len(dataset_tokenized["train"])//(bs*ga_steps)

args = TrainingArguments(
    output_dir="out",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs,
    evaluation_strategy="steps",
    logging_steps=1,
    eval_steps=steps_per_epoch,  # eval and save once per epoch   
    save_steps=steps_per_epoch,
    gradient_accumulation_steps=ga_steps,
    num_train_epochs=epochs,
    lr_scheduler_type="constant",
    optim="paged_adamw_32bit",
    learning_rate=0.0002,
    group_by_length=True,
    fp16=True,
    ddp_find_unused_parameters=False,    # needed for training with accelerate
    report_to="none"
)

**batch size:** As high as possible to increase speed. Consumes VRAM, reduce if OOM.

**gradient_accumulation_steps:** Increases effective batch size without consuming additional VRAM but makes training slower. The effective batch size is batch_size * gradient_accumulation_steps.

**steps_per_epoch:** If your dataset has 80 samples and your effective batch size is 8 (e.g. batch_size 8 and gradient_accumulation_steps 1) you will process your entire dataset in 10 steps (=1 epoch).

**num_train_epochs:** How many epochs to train depends on your dataset. Ideally the loss on your eval split will tell you when to stop training and which checkpoint is the best - but training Guanaco for example results in increasing eval_loss after epoch 2 already, indicating overfitting to the training set, even though the model improves in quality. More on this and an official reply by the QLoRA authors on github and in one of my previous stories.
To sum up: you will simply have to see which checkpoint performs best for 
your specific task. Usually, 3-4 epochs is a good start.

**learning_rate:** We will use the default learning rate suggested by the QLoRA authors, 0.0002 for a 7B (or 13 B) model. For models with more parameters, lower learning rates are suggested: 0.0001 for models with 33B and 65B parameters.

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=collate,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
    args=args,
)

trainer.train()

The following code is a bit different than other scripts (such as the one provided by TheBloke for example) because we added tokens for ChatML before training. We did not change the base model though, this is why before loading the adapters we have to add the new tokens to the base model and tokenizer; otherwise we will try to merge adapters with two additional tokens to a model without these tokens (and this will fail).

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_path="models/Mistral-7B-v0.1"    # input: base model
adapter_path="out/checkpoint-606"     # input: adapters
save_to="models/Mistral-7B-finetuned"    # out: merged model ready for inference

base_model = AutoModelForCausalLM.from_pretrained(
    base_path,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(base_path)

# Add/set tokens (same 5 lines of code we used before training)
tokenizer.pad_token = "</s>"
tokenizer.add_tokens(["<|im_start|>"])
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))
base_model.resize_token_embeddings(len(tokenizer))
base_model.config.eos_token_id = tokenizer.eos_token_id

# Load LoRA adapter and merge
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.merge_and_unload()

model.save_pretrained(save_to, safe_serialization=True, max_shard_size='4GB')
tokenizer.save_pretrained(save_to)